- Прочитайте главы 7-8 из книги "Spark: The Definitive Guide".
- Загрузите датасеты по ссылкам:
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2019
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2020
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2021
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
- Выполните задания.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
                    .master("local")\
                    .appName('SparkLab3')\
                    .getOrCreate()

# Загружаем наборы данных в один DataFrame.
tiktokData19 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2019.csv")
tiktokData20 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2020.csv")
tiktokData21 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2021.csv")
tiktokData22 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2022.csv")


22/12/22 23:06:04 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/12/22 23:06:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/22 23:06:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/22 23:06:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


№1: Добавьте столбец year целочисленного типа, который содержит год датасета для каждой строки данных. Выведите самые танцевальные песни из наборов данных вместе с полем year.

|          track_name|artist_name|year|danceability|
|--------------------|-----------|----|------------|
|  Dancing in My Room|   347aidan|2020|        0.98|
|                 Uno|    Ambjaay|2019|       0.978|
|I'm Looking for M...|   Memetown|2020|       0.977|
|           Hood Baby|       KBFR|2020|       0.974|
|              MICKEY| Lil Yachty|2019|       0.971|
|Muffins In The Fr...|      Tiagz|2020|       0.965|

<div style="text-align: center"> only showing top 6 rows </div>

In [3]:
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import desc
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lit



tiktokData19= tiktokData19.withColumn("year", lit(2019)) 
tiktokData20= tiktokData20.withColumn("year", lit(2020)) 
tiktokData21= tiktokData21.withColumn("year", lit(2021)) 
tiktokData22= tiktokData22.withColumn("year", lit(2022)) 

tiktokData1920 = tiktokData19.unionByName(tiktokData20)
tiktokData192021 = tiktokData1920.unionByName(tiktokData21)
tiktokData = tiktokData192021.unionByName(tiktokData22)

                      
tiktokData.sort(desc("danceability")).select("track_name", "artist_name", "year", "danceability").show(6)

+--------------------+-----------+----+------------+
|          track_name|artist_name|year|danceability|
+--------------------+-----------+----+------------+
|  Dancing in My Room|   347aidan|2020|        0.98|
|                 Uno|    Ambjaay|2019|       0.978|
|I'm Looking for M...|   Memetown|2020|       0.977|
|           Hood Baby|       KBFR|2020|       0.974|
|              MICKEY| Lil Yachty|2019|       0.971|
|Muffins In The Fr...|      Tiagz|2020|       0.965|
+--------------------+-----------+----+------------+
only showing top 6 rows



№2: Найдите исполнителей, которым сопоставлено более одного значения популярности. Выведите эти значения в виде списков.

|          artist|popularity|
|----------------|----------|
|            KYLE|  [65, 66]|
|Kero Kero Bonito|  [57, 56]|
|       Kesh Kesh|  [27, 26]|
|    Mahogany Lox|  [38, 37]|
|   Trevor Daniel|  [66, 65]|
|       blackbear|  [79, 80]|

In [4]:
from pyspark.sql.functions import col
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import array

tiktokData2 = tiktokData\

tiktokData2 = tiktokData2\
.groupBy("artist_name").agg(collect_list("artist_pop").alias("artist_pop"))

tiktokData2.where("artist_name ilike 'K%'").show(50, False)

+------------------+----------------------------+
|artist_name       |artist_pop                  |
+------------------+----------------------------+
|Kanye West        |[90, 90, 90, 90, 90]        |
|Kesha             |[76, 76, 76, 76, 76, 76, 76]|
|Kuraiinu          |[42]                        |
|K CAMP            |[63, 63]                    |
|Kesh Kesh         |[27, 26]                    |
|Kuwada            |[47]                        |
|Kero Kero Bonito  |[56, 57]                    |
|KYLE              |[65, 66, 66, 66]            |
|Killua the Pilgrim|[16]                        |
|Kenny Packer      |[24]                        |
|KRYPTO9095        |[39]                        |
|King Critical     |[29]                        |
|KyleYouMadeThat   |[37]                        |
|Kids Superstars   |[42]                        |
|Kehlani           |[76]                        |
|Kaash Paige       |[61]                        |
|KBFR              |[36]                        |


№3: Исправьте различающиеся значения популярности исполнителей. Замените их на максимальные значения из полученных в предыдущем задании массивов.

|          track_name|     artist_name|               album|artist_pop|
|--------------------|----------------|--------------------|----------|
|     hot girl bummer|       blackbear|     hot girl bummer|        80|
|Hey Julie! (feat....|            KYLE|Light of Mine (De...|        66|
|             Falling|   Trevor Daniel|             Falling|        66|
|             Falling|   Trevor Daniel|            Nicotine|        66|
|            Flamingo|Kero Kero Bonito|            Flamingo|        57|
|       Take Your Man|    Mahogany Lox|       Take Your Man|        38|
|              Vibin'|       Kesh Kesh|              Vibin'|        27|

<div style="text-align: center"> 968 </div>

In [5]:
from pyspark.sql.functions import array_max
from pyspark.sql.functions import when

tiktokData3 = tiktokData\
 
tiktokData2 = tiktokData2.select("artist_name", array_max("artist_pop").alias("artist_pop"))


tiktokData4 = tiktokData3.select("track_name", "artist_name", "album", "artist_pop")
tiktokData3 = tiktokData3.select("track_name", col("artist_name").alias("a"), "album", col("artist_pop").alias("old"))
tiktokData3 = tiktokData3.join(tiktokData2, tiktokData3.a == tiktokData2.artist_name, "left")

tiktokData3 = tiktokData3.select("track_name", "artist_name", "album", "artist_pop")

tiktokData3.where("artist_name ilike 'Tre%'").show(200)

# Выводим различия между новым  и старым датафреймами
substractDf = tiktokData3\
    .subtract(tiktokData4)\
    .select("track_name", "artist_name", "album", "artist_pop")\
    .sort(desc("artist_pop"))

substractDf.show()
tiktokData3.count()

+----------+-------------+--------+----------+
|track_name|  artist_name|   album|artist_pop|
+----------+-------------+--------+----------+
|   Falling|Trevor Daniel| Falling|        66|
|   Falling|Trevor Daniel|Nicotine|        66|
|   Falling|Trevor Daniel| Falling|        66|
|   Falling|Trevor Daniel| Falling|        66|
+----------+-------------+--------+----------+



+--------------------+----------------+--------------------+----------+
|          track_name|     artist_name|               album|artist_pop|
+--------------------+----------------+--------------------+----------+
|             Falling|   Trevor Daniel|            Nicotine|        66|
|Hey Julie! (feat....|            KYLE|Light of Mine (De...|        66|
|            Flamingo|Kero Kero Bonito|            Flamingo|        57|
+--------------------+----------------+--------------------+----------+



968

№4: Сгруппируйте данные по названию трека, названию альбома, имени и популярности певца. Годы датасетов соберите в списки. Популярность треков возьмите из максимального значения. Для столбцов danceability, energy,   speechiness, acousticness, instrumentalness, liveness и valence возьмите среднее значение. Остальные столбцы отбросьте. Сколько получилось строк в сгруппированном DataFrame?

Полученный датафрейм сохраните в новый parquet файл. Это можно сделать методом:

```
df.write.format("parquet").mode("overwrite").save("path to file")
```

Результат при этом сохранится в новую директорию.
Другой способ:

```
df.toPandas().to_parquet("path to file")
```

Результат сохранится в один файл без создания директорий. Для этого способа понадобится библиотека pyarrow.
Установите ее с помощью команд:

```
source ~/Work/bin/activate
pip3 install pyarrow
```

In [7]:
from pyspark.sql.functions import expr
from pyspark.sql.functions import max
from pyspark.sql.functions import avg
from pyspark.sql.functions import size
from pyspark.sql.functions import sort_array
from pyspark.sql.functions import collect_list


tiktokData2 = tiktokData

tiktokData3 = tiktokData2.select("*").groupBy("track_name", "album", "artist_name", "artist_pop")\
.agg(avg("danceability").alias("danceability"), avg("energy").alias("energy"),\
     avg("speechiness").alias("speechiness"), avg("acousticness").alias("acousticness"),\
     avg("instrumentalness").alias("instrumentalness"), avg("liveness").alias("liveness"),\
     avg("valence").alias("valence"), collect_list("year").alias("years"), max("track_pop").alias("track_pop"))\

tiktokData3.show()
tiktokData3.write.format("parquet").mode("overwrite").save("newTick")
tiktokData3.count()

+--------------------+--------------------+--------------------+----------+------------+------+-----------+------------+----------------+--------+-------+------------+---------+
|          track_name|               album|         artist_name|artist_pop|danceability|energy|speechiness|acousticness|instrumentalness|liveness|valence|       years|track_pop|
+--------------------+--------------------+--------------------+----------+------------+------+-----------+------------+----------------+--------+-------+------------+---------+
|"Galactic Mermaid...|"Galactic Mermaid...|            Kuraiinu|        42|       0.621| 0.446|     0.0384|       0.736|             0.0|   0.613|   0.97|      [2019]|        0|
|"Legs (From ""RuP...|"Legs (From ""RuP...|        Lucian Piane|        32|         0.9|  0.72|      0.232|     0.00502|         6.26E-6|   0.543|  0.749|      [2019]|       44|
|"More Than A Woma...|            Greatest|            Bee Gees|        75|       0.632| 0.683|     0.0383|   

853

№5: Отсортируйте песни по количеству вхождений в датасеты и популярности. Выведите их вместе со списками годов наборов данных.

|          track_name|               album|     artist_name|track_pop|               years|
|--------------------|--------------------|----------------|---------|--------------------|
|              Say So|            Hot Pink|        Doja Cat|       80|[2019, 2020, 2021...|
|      Wait a Minute!|        ARDIPITHECUS|          WILLOW|       86|  [2019, 2020, 2022]|
|       Electric Love|            Dopamine|           BØRNS|       82|  [2020, 2021, 2022]|
|           Play Date|Cry Baby (Deluxe ...|Melanie Martinez|       78|  [2020, 2021, 2022]|
|             ROXANNE|             ROXANNE|  Arizona Zervas|       78|  [2019, 2020, 2022]|
|Savage Love (Laxe...|Savage Love (Laxe...|       Jawsh 685|       77|  [2020, 2021, 2022]|

<div style="text-align: center"> only showing top 6 rows </div>

In [8]:
from pyspark.sql.functions import col

tiktokData4 = tiktokData3

tiktokData4.sort(size(col("years")).desc(), col("track_pop").desc()).show(truncate=False)

+--------------------------------------+---------------------------------+-----------------+----------+------------------+-------------------+--------------------+------------+----------------+--------+-------+------------------------+---------+
|track_name                            |album                            |artist_name      |artist_pop|danceability      |energy             |speechiness         |acousticness|instrumentalness|liveness|valence|years                   |track_pop|
+--------------------------------------+---------------------------------+-----------------+----------+------------------+-------------------+--------------------+------------+----------------+--------+-------+------------------------+---------+
|Say So                                |Hot Pink                         |Doja Cat         |88        |0.787             |0.673              |0.159               |0.264       |3.35E-6         |0.0904  |0.779  |[2020, 2021, 2022, 2019]|80       |
|Wait a Minute! 

№6: Найдите все треки с повторяющимися именами. Выполните их сортировку по названию. Выведите названия песен, названия альбомов, имена исполнителей и годы датасетов. Сколько строк в полученной таблице?

|          track_name|               album|   artist_name| years|
|--------------------|--------------------|--------------|------|
|"More Than A Woma...|How Can You Mend ...|      Bee Gees|[2022]|
|"More Than A Woma...|            Greatest|      Bee Gees|[2020]|
|223's (feat. 9lok...|    Melly vs. Melvin|     YNW Melly|[2019]|
|223's (feat. 9lok...|223's (feat. 9lok...|     YNW Melly|[2020]|
|           As It Was|           As It Was|Michael Shynes|[2022]|
|           As It Was|           As It Was|  Harry Styles|[2022]|

<div style="text-align: center"> only showing top 6 rows </div>